In [1]:
!pip install rake-nltk sqlalchemy pymysql pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.5 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()


Saving hashscope-ebf03fb3ec3a.json to hashscope-ebf03fb3ec3a.json


In [3]:
import os

# Ganti 'your-key-file.json' dengan nama file JSON yang diunggah
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "hashscope-ebf03fb3ec3a.json"


In [4]:
!wget https://storage.googleapis.com/cloud-sql-connectors/cloud-sql-proxy/v2.5.0/cloud-sql-proxy.linux.amd64 -O cloud_sql_proxy
!chmod +x cloud_sql_proxy


--2024-12-05 04:59:30--  https://storage.googleapis.com/cloud-sql-connectors/cloud-sql-proxy/v2.5.0/cloud-sql-proxy.linux.amd64
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.207, 142.251.2.207, 2607:f8b0:4023:c0d::cf
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31154279 (30M) [application/octet-stream]
Saving to: ‘cloud_sql_proxy’

cloud_sql_proxy     100%[===================>]  29.71M   195MB/s    in 0.2s    

2024-12-05 04:59:30 (195 MB/s) - ‘cloud_sql_proxy’ saved [31154279/31154279]



In [5]:
!nohup ./cloud_sql_proxy hashscope:asia-southeast2:dbhashscope --port=3306 --credentials-file=hashscope-ebf03fb3ec3a.json > proxy.log 2>&1 &


In [6]:
!cat proxy.log


2024/12/05 04:59:30 Authorizing with the credentials file at "hashscope-ebf03fb3ec3a.json"
2024/12/05 04:59:31 [hashscope:asia-southeast2:dbhashscope] Listening on 127.0.0.1:3306
2024/12/05 04:59:31 The proxy has started successfully and is ready for new connections!


In [8]:
from rake_nltk import Rake
import pandas as pd
from sqlalchemy import create_engine

# === KONFIGURASI DATABASE ===
db_user = "root"
db_password = "123456"
db_name = "db_hashscope"
db_ip = "127.0.0.1"
db_port = 3306

# Buat URL koneksi
db_connection_url = f"mysql+pymysql://{db_user}:{db_password}@{db_ip}:{db_port}/{db_name}"

# Buat koneksi SQLAlchemy
engine = create_engine(db_connection_url)



In [9]:
try:
    with connection.cursor() as cursor:
        cursor.execute("SHOW TABLES;")
        tables = cursor.fetchall()
        print("Tabel dalam database:", tables)
except Exception as e:
    print(f"Terjadi kesalahan saat menjalankan query: {e}")


Terjadi kesalahan saat menjalankan query: name 'connection' is not defined


In [20]:
!pip install rake-nltk -q

In [21]:
from rake_nltk import Rake

In [25]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [89]:
# Inisialisasi RAKE dengan batas panjang frasa
r = Rake(punctuations=['.', ',', ';', ':', '"', '!', '?', ']', '[', '(', ')', '...', '-'], max_length=7)  # Batasi hingga 3 kata


In [90]:
def fetch_data():
    query = """
    SELECT idx_datasets AS id, full_text FROM x_datasets
    UNION
    SELECT idweb_datasets AS id, full_text FROM web_datasets
    UNION
    SELECT idyoutube_datasets AS id, full_text FROM youtube_datasets;
    """
    with engine.connect() as conn:
        df = pd.read_sql(query, conn)
    print(f"Data berhasil diambil: {len(df)} baris")
    return df

# Ambil data dari database
data = fetch_data()


Data berhasil diambil: 204 baris


In [91]:
import re

def preprocess_text(text):
    # Hapus mention seperti @username
    text = re.sub(r"@\w+", "", text)
    # Hapus link yang mengandung https atau www
    text = re.sub(r"http\S+|www\.\S+", "", text)
    # Hapus spasi ekstra yang mungkin muncul setelah penghapusan
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [92]:
# Preprocessing teks untuk menghapus kata dengan @
print("Melakukan preprocessing teks...")
data["preprocessed_text"] = data["full_text"].apply(preprocess_text)
print("Preprocessing selesai.")


Melakukan preprocessing teks...
Preprocessing selesai.


In [96]:
# Fungsi untuk mengekstraksi kata kunci menggunakan RAKE
def extract_keywords_with_scores(text):
    r.extract_keywords_from_text(text)
    return r.get_ranked_phrases_with_scores()  # Skor dan frasa

# Terapkan RAKE ke teks yang sudah diproses
print("Melakukan ekstraksi kata kunci dengan RAKE...")
data["rake_results"] = data["preprocessed_text"].apply(extract_keywords_with_scores)

# Ubah hasil menjadi DataFrame terpisah
rake_results = []
for idx, row in data.iterrows():
    for score, phrase in row["rake_results"]:
        rake_results.append({"id": row["id"], "score": score, "phrase": phrase})

rake_df = pd.DataFrame(rake_results)
filtered_rake_df = rake_df[rake_df["score"] > 10]

print("Ekstraksi selesai.")


Melakukan ekstraksi kata kunci dengan RAKE...
Ekstraksi selesai.


In [97]:
# Simpan hasil RAKE ke CSV
csv_path = "rake_results.csv"
filtered_rake_df.to_csv(csv_path, index=False)
print(f"Hasil RAKE disimpan sementara ke {csv_path}.")


Hasil RAKE disimpan sementara ke rake_results.csv.


In [98]:
# Tampilkan 20 data pertama setelah filter
print("20 Data Setelah Filter (Skor > 10):")
display(filtered_rake_df.head(20))


20 Data Setelah Filter (Skor > 10):


,id,score,phrase
0,103,49.0,pakai teknologi jaman perang dunia hairan aku
1,105,49.0,kayaknya yg ini sdh pakai teknologi canggih
2,105,36.0,biaya pembuatannya gak masuk mumgkin bro
3,105,25.0,bukan kayak hasil printer warna
4,107,16.0,masuk kuliah wajib militer
5,108,25.0,berarti ini masalah gelombang sinyal
6,109,23.5,d4 supervisor jaminan mutu pangan
7,109,23.5,d4 supervisor jaminan mutu pangan
20,110,49.0,1 dan tingkatkan kualitas grafis game anda
21,110,16.0,# idncitizen # promedia


**SQL QUERY UNTUK UPDATE KOLOM FREKUENSI**

UPDATE main_topics
SET frequency = (
    SELECT COUNT(*)
    FROM main_topics AS mt
    WHERE mt.topics_name = main_topics.topics_name
);



In [101]:
def insert_main_topics_to_db_with_check(filtered_rake_df):
    print("Memulai proses insert data ke database...")
    try:
        # Ambil ID yang sudah ada di tabel main_topics
        with engine.connect() as conn:
            existing_ids_query = "SELECT x_datasets_idx_datasets FROM main_topics WHERE x_datasets_idx_datasets IS NOT NULL"
            existing_ids = pd.read_sql(existing_ids_query, conn)["x_datasets_idx_datasets"].tolist()
            print(f"Jumlah ID yang sudah ada: {len(existing_ids)}")

        # Filter data baru agar hanya memasukkan ID yang belum ada
        new_data = filtered_rake_df[~filtered_rake_df["id"].isin(existing_ids)]
        print(f"Jumlah data baru yang akan di-insert: {len(new_data)}")

        # Siapkan data untuk insert
        data_to_insert = []
        for _, row in new_data.iterrows():
            data_to_insert.append({
                "topics_name": row["phrase"],               # Nama topik dari hasil RAKE
                "frequency": 1,                             # Frekuensi default adalah 1
                "x_datasets_idx_datasets": row["id"],       # Isi ID dari x_datasets
                "web_datasets_idweb_datasets": None,        # NULL untuk web_datasets
                "youtube_datasets_idyoutube_datasets": None # NULL untuk youtube_datasets
            })

        # Konversi ke DataFrame jika ada data baru
        if data_to_insert:
            df_to_insert = pd.DataFrame(data_to_insert)

            # Insert data baru ke database
            with engine.connect() as conn:
                df_to_insert.to_sql("main_topics", con=conn, if_exists="append", index=False)
            print("Data berhasil disimpan ke database.")
        else:
            print("Tidak ada data baru untuk di-insert.")

    except Exception as e:
        print("Kesalahan saat menyimpan data ke database:")
        print(e)

# Panggil fungsi untuk insert data dengan pengecekan
insert_main_topics_to_db_with_check(filtered_rake_df)


Memulai proses insert data ke database...
Jumlah ID yang sudah ada: 158
Jumlah data baru yang akan di-insert: 0
Tidak ada data baru untuk di-insert.
